In [7]:
import requests
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

## Crawl data from Goodreads

In [7]:
goodread_df = pd.read_csv(
    r'D:\Online_Learning\Practical_DL\final_project\books.csv', on_bad_lines='skip')

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor, as_completed


def fetch_data_for_isbn(isbn):
    return get_reviews_from_isbn(url, isbn)


def get_reviews_from_isbn(url, isbn):
    full_url = url+f"{isbn}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Retry mechanism
    for attempt in range(3):  # Retry up to 3 times
        try:
            response = requests.get(
                full_url, headers=headers, allow_redirects=True, timeout=10)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                break
        except requests.RequestException as e:
            print(f"Error fetching data for ISBN {isbn}: {e}")
            time.sleep(2 ** attempt)  # Exponential backoff
        else:
            print(
                f"Failed to fetch data after {attempt + 1} attempts. Status Code: {response.status_code}")
            return pd.DataFrame()  # Return empty DataFrame if unsuccessful

    # Parsing logic as before
    reviews = soup.find_all('article', class_='ReviewCard')
    all_reviews = []

    for review in reviews:
        # Modularize parsing into a function
        review_data = parse_review(review, isbn)
        all_reviews.append(review_data)

    return pd.DataFrame(all_reviews)


def parse_review(review, isbn):
    try:
        reviewer_name = review.find(
            'div', {'data-testid': 'name'}).get_text(strip=True)
        rating_section = review.find('div', class_='ShelfStatus')
        rating = rating_section.find('span', {'role': 'img'}).get(
            'aria-label', '').split()[1]
        comment_section = review.find(
            'div', {'data-testid': 'contentContainer'})
        comment = comment_section.get_text(
            strip=True) if comment_section else "No comment provided"
    except Exception as e:
        print(f"Error parsing review for ISBN {isbn}: {e}")
        return {'isbn': isbn, 'reviewer': '', 'rating': '', 'comment': ''}

    return {
        'isbn': isbn,
        'reviewer': reviewer_name,
        'rating': rating,
        'comment': comment
    }


def fetch_reviews_parallel(isbn_list):
    reviews_df = []
    # Reduced number of workers to lessen the load
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_isbn = {executor.submit(
            fetch_data_for_isbn, isbn): isbn for isbn in isbn_list}
        for future in as_completed(future_to_isbn):
            isbn = future_to_isbn[future]
            try:
                data = future.result()
                reviews_df.append(data)
                print(f"Data fetched for ISBN {isbn} with shape {data.shape}")
                time.sleep(1)  # Throttle requests
            except Exception as exc:
                print(f"{isbn} generated an exception: {exc}")
    return reviews_df

In [ ]:
url = f"https://www.goodreads.com/book/isbn/"
isbn_list = goodread_df['isbn'].unique()

reviews_dataframes = fetch_reviews_parallel(isbn_list)

In [35]:
test = pd.concat(reviews_dataframes)
test = test[test['reviewer'] != ""]
test.to_csv('reviews_goodread.csv', index=False)

In [36]:
test = pd.read_csv(
    r'D:\Online_Learning\Practical_DL\final_project\reviews_goodread.csv')
test.shape

(273442, 4)

In [37]:
test.head()

,isbn,reviewer,rating,comment
0,0439358078,Jayson,5,(A) 86%| ExtraordinaryNotes:An angsty apprehen...
1,0439358078,Navessa,5,"Seriously, don't read this review if you haven..."
2,0439358078,Diane ϟ [ Lestrange ],5,Interview with JK Rowling...Stephen Fry:Can we...
3,0439358078,Jayson,5,(A) 86%| ExtraordinaryNotes:It's a transitiona...
4,0439358078,Hannah Azerang,5,I had to re read it. I was in such a nostalgic...


## Crawl news from ABC

In [4]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from urllib.robotparser import RobotFileParser
import random
import logging
import numpy as np

# Define a custom logging handler to capture log records in a list
class ListHandler(logging.Handler):
    def __init__(self):
        super().__init__()
        self.log_records = []

    def emit(self, record):
        self.log_records.append(record)

# Set up basic logging configuration
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Create an instance of the custom handler and add it to the root logger
list_handler = ListHandler()
logging.getLogger().addHandler(list_handler)

# User-agent rotation
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.1 Safari/605.1.15',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:85.0) Gecko/20100101 Firefox/85.0',
]

def check_robots_txt(url):
    rp = RobotFileParser()
    rp.set_url(url + "/robots.txt")
    rp.read()
    return rp

def get_links_from_sgtimes(category_sgt, headers, num_limit, rp):
    full_url = "https://thesaigontimes.vn" + category_sgt

    if not rp.can_fetch(headers['User-Agent'], full_url):
        logging.warning(
            f"Fetching from {full_url} is disallowed by robots.txt")
        return []

    response = requests.get(full_url, headers=headers, timeout=10)
    soup = BeautifulSoup(response.text, 'html.parser')

    links = soup.find_all(
        'h3', class_='entry-title td-module-title', limit=num_limit)
    href_link = []
    for link in links:
        try:
            href_link.append(link.find('a')['href'])
            time.sleep(1)
        except:
            logging.error("Not Found")

    return list(set(href_link))

def get_links_from_cnn(category, headers, num_limit, rp):
    full_url = "https://edition.cnn.com" + category

    if not rp.can_fetch(headers['User-Agent'], full_url):
        logging.warning(
            f"Fetching from {full_url} is disallowed by robots.txt")
        print(f"Fetching from {full_url} is disallowed by robots.txt")
        return []

    response = requests.get(full_url, headers=headers, timeout=10)
    soup = BeautifulSoup(response.text, 'html.parser')

    links = set()
    page_number = 1

    while len(links) < num_limit:
        response = requests.get(full_url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')

        for item in soup.find_all('a', class_='container__link'):
            links.add("https://edition.cnn.com" + item['href'])
            if len(links) >= num_limit:
                break

        page_number += 1
        time.sleep(1)  # Throttle requests

    return list(links)[:num_limit]

def get_links_from_bbc(category, headers, num_limit, rp):
    full_url = "https://www.bbc.com" + category

    if not rp.can_fetch(headers['User-Agent'], full_url):
        logging.warning(
            f"Fetching from {full_url} is disallowed by robots.txt")
        print(f"Fetching from {full_url} is disallowed by robots.txt")
        return []

    response = requests.get(full_url, headers=headers, timeout=10)
    soup = BeautifulSoup(response.text, 'html.parser')

    links = soup.find_all('a', {'data-testid':'internal-link'}, limit=num_limit)
    href_link = []
    for link in links:
        try:
            href_link.append("https://www.bbc.com" + link['href'])
            time.sleep(1)
        except:
            logging.error("Not Found")

    return list(set(href_link))

def get_text_from_link(url, headers, failed_links):
    logging.info(f"Fetching from: {url}")
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            logging.error(
                f"Failed to fetch data from {url}, status code: {response.status_code}")
            failed_links.append(url)
            return np.nan
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text_content = ' '.join(paragraph.get_text(strip=True) for paragraph in paragraphs)
        logging.debug(f"Fetched content from {url}: {text_content[:200]}...")  # Log first 200 characters
        return text_content
    except Exception as e:
        logging.error(f"Exception occurred while fetching from {url}: {e}")
        failed_links.append(url)
        return np.nan

def get_texts_from_links_parallel(links, headers):
    failed_links = []
    results_dict = {}
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {executor.submit(
            get_text_from_link, url, headers, failed_links): url for url in links}
        for future in as_completed(future_to_url):
            url = future_to_url[future]
            try:
                result = future.result()
                results_dict[url] = result
                logging.info(f"Completed fetching from: {url}")
                # Random delay to throttle requests
                time.sleep(random.uniform(1, 3))
            except Exception as exc:
                logging.error(f"{url} generated an exception: {exc}")
                failed_links.append(url)
                results_dict[url] = np.nan
    results = [results_dict[url] for url in links]
    return results, failed_links

def run_parallel_processing(links, headers):
    texts, failed_links = get_texts_from_links_parallel(links, headers)
    df = pd.DataFrame({'url': links, 'txt': texts})
    return df, failed_links

if __name__ == "__main__":
    category_cnn = '/business'  # Example category for CNN
    category_sgt = '/tai-chinh-ngan-hang'  # Example category for Saigon Times
    category_bbc = '/innovation'  # Example category for BBC

    headers = {
        'User-Agent': random.choice(user_agents)
    }
    num_limit = 10

    # Check robots.txt
    cnn_rp = check_robots_txt("https://edition.cnn.com")
    sgt_rp = check_robots_txt("https://thesaigontimes.vn")
    bbc_rp = check_robots_txt("https://www.bbc.com")

    links_cnn = get_links_from_cnn(category_cnn, headers, num_limit, cnn_rp)
    links_sgt = get_links_from_sgtimes(category_sgt, headers, num_limit, sgt_rp)
    links_bbc = get_links_from_bbc(category_bbc, headers, num_limit, bbc_rp)

    df_txt, failed_links = run_parallel_processing(links_sgt+links_cnn+links_bbc, headers)
    df_txt.to_csv('scraped_texts.csv', index=False)

    # # Log failed links
    # if failed_links:
    #     logging.info(
    #         f"Failed to fetch texts from the following links: {failed_links}")
    # else:
    #     logging.info("Successfully fetched texts from all links.")

    # # Retrieve error log content
    # error_logs = [record.getMessage(
    # ) for record in list_handler.log_records if record.levelno == logging.ERROR]
    # print("Error Logs:\n", "\n".join(error_logs))


2024-09-18 12:39:46,544 - INFO - Fetching from: https://thesaigontimes.vn/de-xuat-ho-tro-tai-chinh-linh-hoat-cho-dia-phuong-bi-anh-huong-bao/
2024-09-18 12:39:46,547 - INFO - Fetching from: https://thesaigontimes.vn/vang-nhan-tung-buoc-xo-do-cac-ky-luc-ve-gia/
2024-09-18 12:39:46,552 - INFO - Fetching from: https://thesaigontimes.vn/ty-gia-bien-dong-kho-luong-he-qua-nao-cho-doanh-nghiep/
2024-09-18 12:39:46,556 - INFO - Fetching from: https://thesaigontimes.vn/thong-tu-12-va-viec-coi-troi-tin-dung-tieu-dung/
2024-09-18 12:39:46,559 - INFO - Fetching from: https://thesaigontimes.vn/facebook-tiktok-va-loat-ong-lon-cong-nghe-nop-thue-hon-6-200-ti-dong-trong-8-thang/
2024-09-18 12:39:46,560 - INFO - Fetching from: https://thesaigontimes.vn/doanh-nghiep-nguoi-dan-co-the-duoc-vay-von-khong-lai-suat-de-khoi-phuc-san-xuat/
2024-09-18 12:39:46,562 - INFO - Fetching from: https://edition.cnn.com/2024/09/17/food/lunchly-logan-paul-mrbeast/index.html
2024-09-18 12:39:46,565 - INFO - Fetching from:

In [5]:
test = pd.read_csv('scraped_texts.csv')
test.head()

,url,txt
0,https://thesaigontimes.vn/de-xuat-ho-tro-tai-c...,Kinh tế Sài Gòn Online Kinh tế Sài Gòn Online ...
1,https://thesaigontimes.vn/vang-nhan-tung-buoc-...,Kinh tế Sài Gòn Online Kinh tế Sài Gòn Online ...
2,https://thesaigontimes.vn/ty-gia-bien-dong-kho...,Kinh tế Sài Gòn Online Kinh tế Sài Gòn Online ...
3,https://thesaigontimes.vn/thong-tu-12-va-viec-...,Kinh tế Sài Gòn Online Kinh tế Sài Gòn Online ...
4,https://thesaigontimes.vn/facebook-tiktok-va-l...,Kinh tế Sài Gòn Online Kinh tế Sài Gòn Online ...


In [7]:
print(test['url'][12])
print()
print(test['txt'][12])

https://edition.cnn.com/2024/09/17/economy/us-retail-sales-august/index.html

Markets Hot Stocks Fear & Greed Index Latest Market News Hot Stocks A key driver of the US economy remains solid. Spending at US retailers rose 0.1% in August from the prior month, the Commerce Department reported Tuesday. That’s a much slower pace than July’s upwardly revised 1.1% gain, but well above the 0.2% decline economists projected in a FactSet poll. The figures are adjusted for seasonal swings but not inflation. It’s an encouraging sign for America’s economy, since consumer spending represents two-thirds of US economic output. Retail sales make up a sizable chunk of overall spending. Tuesday’s report is the final major economic release beforethe Federal Reserve announces its latest interest-rate moveon Wednesday. The numbers do little to influence the size of the expected rate cut. The debate over whether the Fed will roll out a quarter-point rate cut, or a larger, half-point cut has intensified rece

## Crawl with Reuters

In [2]:
import requests

def check_robots_txt(url):
    robots_url = f"{url}/robots.txt"
    response = requests.get(robots_url)
    if response.status_code == 200:
        print(f"Robots.txt content for {url}:")
        print(response.text)
    else:
        print(f"No robots.txt found for {url}")


# Example usage
check_robots_txt("https://edition.cnn.com/")

Robots.txt content for https://edition.cnn.com/:
Sitemap: https://www.cnn.com/sitemaps/cnn/index.xml
Sitemap: https://www.cnn.com/sitemaps/cnn/news.xml
Sitemap: https://www.cnn.com/sitemap/news.xml
Sitemap: https://www.cnn.com/sitemaps/sitemap-section.xml
Sitemap: https://www.cnn.com/sitemaps/sitemap-interactive.xml
Sitemap: https://www.cnn.com/ampstories/sitemap.xml
Sitemap: https://edition.cnn.com/sitemaps/news.xml
Sitemap: https://www.cnn.com/sitemap/article/cnn-underscored.xml
Sitemap: https://www.cnn.com/sitemap/section/cnn-underscored.xml
Sitemap: https://www.cnn.com/cnn-underscored/money/sitemap.xml
Sitemap: https://www.cnn.com/cnn-underscored/money/sitemap-news.xml
Sitemap: https://www.cnn.com/sitemap/section/politics.xml
Sitemap: https://www.cnn.com/sitemap/article/opinions.xml
Sitemap: https://www.cnn.com/sitemap/article.xml
Sitemap: https://www.cnn.com/sitemap/section.xml
Sitemap: https://www.cnn.com/sitemap/video.xml
Sitemap: https://www.cnn.com/sitemap/gallery.xml
Sitemap:

In [16]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from urllib.robotparser import RobotFileParser
import random
import logging

# User-agent rotation
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.1 Safari/605.1.15',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:85.0) Gecko/20100101 Firefox/85.0',
]

headers = {
    'User-Agent': random.choice(user_agents)
}
num_limit = 10

In [58]:
def check_robots_txt(url):
    rp = RobotFileParser()
    rp.set_url(url + "/robots.txt")
    rp.read()
    return rp

def get_links_from_reuters(category, headers, num_limit, rp):
    full_url = "https://www.bbc.com/" + category

    if not rp.can_fetch(headers['User-Agent'], full_url):
        logging.warning(
            f"Fetching from {full_url} is disallowed by robots.txt")
        print(f"Fetching from {full_url} is disallowed by robots.txt")
        return []

    response = requests.get(full_url, headers=headers, timeout=10)
    soup = BeautifulSoup(response.text, 'html.parser')

    links = soup.find_all('a', {'data-testid':'internal-link'}, limit=10)
    href_link = []
    for link in links:
        try:
            href_link.append("https://www.bbc.com" + link['href'])
            time.sleep(1)
        except:
            logging.error("Not Found")

    return list(set(href_link))


bbc_rp = check_robots_txt("https://www.bbc.com/")
links_bbc = get_links_from_reuters('innovation', headers, 10, bbc_rp)

In [59]:
links_bbc

['https://www.bbc.com/video',
 'https://www.bbc.com/news/technology-53476117',
 'https://www.bbc.com/sport',
 'https://www.bbc.com/future/article/20240912-the-archivists-battling-to-save-the-internet',
 'https://www.bbc.com/',
 'https://www.bbc.com/future/article/20240912-the-12-stop-flight-across-the-world-in-a-60-year-old-plane',
 'https://www.bbc.com/news/articles/cp3dv0ddwp7o',
 'https://www.bbc.com/news/articles/c5y3y79llndo']

In [53]:
import requests
from bs4 import BeautifulSoup

html_test = '''<div data-testid="anchor-inner-wrapper"><a href="/future/article/20240912-the-archivists-battling-to-save-the-internet" data-testid="internal-link" class="sc-2e6baa30-0 gILusN"><div data-testid="london" class="sc-f98732b0-2 bNZLNs"><div data-testid="card-media-wrapper" class="sc-f98732b0-4 emSCvS"><div data-testid="card-media" class="sc-f98732b0-1 jSgXDY"><div class="sc-814e9212-1 fcEyBx"><img sizes="(min-width: 768px) 50vw, 100vw" srcset="https://ichef.bbci.co.uk/images/ic/160x90/p0jqfff7.jpg.webp 160w,https://ichef.bbci.co.uk/images/ic/240x135/p0jqfff7.jpg.webp 240w,https://ichef.bbci.co.uk/images/ic/320x180/p0jqfff7.jpg.webp 320w,https://ichef.bbci.co.uk/images/ic/480x270/p0jqfff7.jpg.webp 480w,https://ichef.bbci.co.uk/images/ic/640x360/p0jqfff7.jpg.webp 640w,https://ichef.bbci.co.uk/images/ic/800x450/p0jqfff7.jpg.webp 800w,https://ichef.bbci.co.uk/images/ic/1024x576/p0jqfff7.jpg.webp 1024w,https://ichef.bbci.co.uk/images/ic/1376x774/p0jqfff7.jpg.webp 1376w,https://ichef.bbci.co.uk/images/ic/1920x1080/p0jqfff7.jpg.webp 1920w" src="https://ichef.bbci.co.uk/images/ic/480x270/p0jqfff7.jpg.webp" loading="eager" alt="Art of laptop breaking into pieces (Credit: Serenity Strull/ Getty Images)" class="sc-814e9212-0 hIXOPW"></div></div></div><div data-testid="card-text-wrapper" class="sc-f98732b0-5 gGpWNz"><div class="sc-4fedabc7-1 kbvCmj"><div class="sc-4fedabc7-0 kZtaAl"><h2 data-testid="card-headline" class="sc-4fedabc7-3 dsoipF">The web is disappearing. Can the Internet Archive save it?<!-- --></h2></div></div><p data-testid="card-description" class="sc-f98732b0-0 iQbkqW">Research shows 25% of web pages posted between 2013 and 2023 have vanished. A few organisations are racing to save the echoes of the web, but new risks threaten their very existence.</p><div class="sc-4e537b1-0 hJDQRX"><span data-testid="card-metadata-lastupdated" class="sc-4e537b1-1 dsUUMv">14 hrs ago</span><div data-testid="card-metadata-separator" class="sc-4e537b1-3 lmDmEx"></div><span data-testid="card-metadata-tag" class="sc-4e537b1-2 eRsxHt">Future</span></div><div data-testid="card-relatedUrls" class="sc-c01f71b3-0 ghsqpk"></div></div></div></a></div>'''
soup = BeautifulSoup(html_test, 'html.parser')
link = soup.find('a')['href']
print(link)

/future/article/20240912-the-archivists-battling-to-save-the-internet


In [55]:
response = requests.get("https://www.bbc.com/innovation", headers=headers, timeout=10)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all('a', {'data-testid':'internal-link'}, limit=10):
    print(link['href'])

/
/
/sport
/video
/future/article/20240912-the-archivists-battling-to-save-the-internet
/news/technology-53476117
/news/articles/cp3dv0ddwp7o
/future/article/20240912-the-12-stop-flight-across-the-world-in-a-60-year-old-plane
/news/technology-53476117
/news/articles/c5y3y79llndo


In [9]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

## Base Model
MODEL_NAME = r"D:\Online_Learning\Practical_DL\bart_large_cnn"

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model.eval()

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to generate summary
def generate_summary(model, tokenizer, text, max_length=400, min_length=130):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    inputs = inputs.to(device)

    # Generate summary
    summary_ids = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        num_beams=4,
        early_stopping=True,
        max_length=max_length,
        min_length=min_length,
    )

    # Decode the generated summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [10]:
generated_summaries = [generate_summary(model, tokenizer, text) for text in test['txt']]
test['summaried'] = generated_summaries

d:\Programs\Anaconda\envs\pytorch\lib\site-packages\transformers\models\bart\modeling_bart.py:603: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [11]:
len(generated_summaries)

24

In [12]:
import textwrap

for i in range(len(generated_summaries)):
    print(f"Original Text: {textwrap.wrap(test['url'][i], width=100)}")
    print("Generated Summary:")
    summary = test['summaried'][i]
    paragraphs = summary.split('\n')
    for paragraph in paragraphs:
        lines = textwrap.wrap(paragraph, width=100)
        for line in lines:
            print(line)
    print("\n")

Original Text: ['https://thesaigontimes.vn/de-xuat-ho-tro-tai-chinh-linh-hoat-cho-dia-phuong-bi-anh-huong-bao/']
Generated Summary:
Bộ Tài chính sẽ xem xét và đề xuất phương án hỗ trợ tài. Bão do bão và hoàn lưu bão, ăn bị thiệt hại
cho các  các  ĉnh giá lửa 2024. Các āXVN đã thực hiỉn đông cân  năm 2024. Trong tổng số 26
 ĉn hưởng, có 12  ‘’’ ‘”’, “”, ” ”   ,  h”.


Original Text: ['https://thesaigontimes.vn/vang-nhan-tung-buoc-xo-do-cac-ky-luc-ve-gia/']
Generated Summary:
Kinh tế Sài Gòn Online (KTSG Online) – Kim loại quý toàn cục mới. Giá vàng cũng duy trì vào sáng hôm
nay (15-9) Tỉnh rõ ràng về lãi suất. Hồng niễm yἿt giá  vàng nhẫn 77,88 – 79,08 triệu  mua  bán ra.


Original Text: ['https://thesaigontimes.vn/ty-gia-bien-dong-kho-luong-he-qua-nao-cho-doanh-nghiep/']
Generated Summary:
Kinh tế Sài Gòn Online (KTSG) – Trong bối cảnh đô la Mỹ, giá trở lại trong hai tháng qua. Ngoài ra,
sự biỉn động khó lường của yen Nhật cũng công cây lớn. 27,7% là mức tăng trưng ứng vồng và tơn
124